## 요약
- 기존 전처리 코드에서는 정수 인코딩한 token들도 scaling을 진행함
- transformer에서 embedding model을 사용하려면 정수 값으로 입력 받아야 함
- 또한 중고 도서 데이터가 아닌 도서 정보만 사용
- 이에 맞게 코드 수정

In [1]:
import os, natsort, re
from tqdm import tqdm
import time, random

In [2]:
from itertools import repeat, chain

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import os

from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error as mape
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_squared_log_error as msle

import matplotlib_inline.backend_inline
matplotlib_inline.backend_inline.set_matplotlib_formats("png2x")
# 테마 설정: "default", "classic", "dark_background", "fivethirtyeight", "seaborn"
mpl.style.use("fivethirtyeight")
# 이미지가 레이아웃 안으로 들어오도록 함
mpl.rcParams.update({"figure.constrained_layout.use": True})
mpl.rcParams['axes.unicode_minus'] = False

In [3]:
from google.colab import drive

drive.mount('/content/drive/', force_remount=True)

ModuleNotFoundError: No module named 'google.colab'

In [4]:
#cd /content/drive/MyDrive/AI3_prjct2_aladin/

In [4]:
cd /content/drive/MyDrive/WASSUP-ESTsoft-AI/project/project2/

[Errno 2] No such file or directory: '/content/drive/MyDrive/WASSUP-ESTsoft-AI/project/project2/'
/home/doeun/code/AI/ESTSOFT2024/workspace/2.project_text/transformer_models/research


In [3]:
# 로컬에서

plt.rc("font", family = "D2Coding")
plt.rcParams["axes.unicode_minus"] = False

In [4]:
PRJCT_PATH = '/home/doeun/code/AI/ESTSOFT2024/workspace/2.project_text/aladin_book_price/'
#PRJCT_PATH = '/content/drive/MyDrive/WASSUP-ESTsoft-AI/project/project2/'
#PRJCT_PATH = '/content/drive/MyDrive/AI3_prjct2_aladin/aladin_usedbook/'
save_dir = 'processed/model_input'
dir_path = os.path.join(PRJCT_PATH,save_dir)
#dir_path = './'

In [5]:
ls

20250211_reg_simulation.ipynb:Zone.Identifier
240710_crawling_step0.ipynb
240711_crawling_step1.ipynb
240711_preprocess_bookinfo.ipynb
240715_encoding_usedinfo.ipynb
240716_check_bookinfo.ipynb
240716_check_bookinfo2.ipynb
240716_encoding_bookinfo.ipynb
240717_simple_model_for_sample.ipynb
240717_split_and_scale.ipynb
240718_step0_by_js.ipynb
240719_additional_eda.ipynb
240719_simple_model_for_cropped.ipynb
240721_GridSearch_for_XGB.ipynb
240721_experiment_w_XGB.ipynb
240721_hyperparameters_XGB.ipynb
241023_basic_model.ipynb
241023_preprocess_for_pred_rglr.ipynb
241024_basic_torch_model.ipynb
250112_modulized_test_20480_init_lr_2.12.ipynb
250112_modulized_test_20480_init_lr_2.12.ipynb:Zone.Identifier
250112_modulized_test_20480_init_lr_analyze2.ipynb
250112_modulized_test_20480_init_lr_analyze2.ipynb:Zone.Identifier
250112_modulized_test_plot_rslt.ipynb
250112_modulized_test_plot_rslt.ipynb:Zone.Identifier
250123_modulized_test_20480_init_lr_test_score.ipynb
250123_modulized_test_20480

In [6]:
import sys
sys.path.append(PRJCT_PATH)

In [7]:
RSLT_DIR = PRJCT_PATH + 'processed/'

bookinfo_name = 'bookinfo_ver{}.pkl'.format(0.75)
bookinfo_path = os.path.join(RSLT_DIR,bookinfo_name)

sys.path.append(PRJCT_PATH)
from module_aladin.file_io import load_pkl, save_pkl
from module_aladin.data_process import pd_datetime_2_datenum

from konlpy.tag import Mecab
from tensorflow.keras.preprocessing.sequence import pad_sequences
import itertools
from sklearn.preprocessing import MinMaxScaler

2025-03-11 12:26:07.903875: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-11 12:26:08.408857: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-11 12:26:08.536394: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-03-11 12:26:08.536424: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not ha

In [8]:
def set_corpus_size(freq,size_feat,mode):
    # 입력받은 mode와 size_feat에 따라 size 크기 결정
    if mode == 'uniform':
        cond = freq['counts']>=freq['counts'].iloc[size_feat]
        size = np.sum(cond)
    elif mode =='ths':
        cond = freq[freq['counts'] > size_feat]
        size = np.sum(cond)
    else :
        if size_feat == None : size = len(freq)
#        elif size_feat > len(data) : size = len(freq)
        else : size = size_feat
    return size

def make_encoding_by_freq(freq,null_val='[PAD]',size_feat=None,mode=None):
    #빈도수 기반 정수 인코딩 dict 만들기
    # freq : token 별 등장 빈도 (value_count), size_feat : size관련 변수(max_size, ths등), mode : size 결정 방법
    df_freq = pd.DataFrame(freq).T
    df_freq = df_freq.rename(columns={0:'token',1:'counts'})
    temp = df_freq.sort_values(by='counts',ascending=False)
    size = set_corpus_size(temp,size_feat,mode)
    temp = temp.iloc[:size]
    temp['val'] = np.arange(size)+1
    temp2 = temp.set_index('token').to_dict()
    map_token_encode = temp2['val']
    map_token_encode[null_val]=0
    return map_token_encode

def encode_tokens(map_token,x,oov=True):
    oov_val = len(map_token)+1 if oov else 0 
    return map_token[x] if x in map_token else oov_val

def make_author_encode_map(bookinfo,ths_author):
    pvtb = pd.pivot_table(data=bookinfo,index='Author',values='SalesPoint',aggfunc=np.sum)
    pvtb = pvtb.sort_values(by='SalesPoint',ascending=False)
    author_top_k= pvtb[pvtb['SalesPoint']>=ths_author].index
    encode_author = pd.DataFrame({'author' : author_top_k.values,'val':np.arange(1,len(author_top_k)+1)})
    encode_author = encode_author.set_index('author')
    return encode_author.to_dict()['val']

def make_publshr_encode_map(publshr_data,ths_publshr):
    stats = publshr_data.value_counts().sort_values(ascending=False)
    top_k_val = stats.iloc[ths_publshr]
    publshr_top_k = list(stats[stats >= top_k_val].index)
    return {
        publshr : n+1
        for n,publshr in enumerate(publshr_top_k)
    }
    

def make_store_encode_map(store_data):
    stores= store_data.value_counts().sort_values(ascending=False)
    return {
        place : n+1
        for n,place in enumerate(stores.index)
    }

In [9]:
file_name = 'bookinfo_ver{}.csv'.format(1.0)
file_path = os.path.join(RSLT_DIR,file_name)
bookinfo_raw = pd.read_csv(file_path)

In [10]:
cols = bookinfo_raw.columns.to_list()
x_idxs, y_idx = [0,2,3,4,5,6,9,10], 7
x_cols = [cols[i] for i in x_idxs]
y_col = cols[y_idx]

data_X, data_y  = bookinfo_raw[x_cols], bookinfo_raw[y_col]

In [11]:
from sklearn.model_selection import train_test_split

X_data, X_tst, y_data, y_tst = train_test_split(data_X,data_y,test_size=0.2,random_state=329)
X_trn, X_vld, y_trn, y_vld = train_test_split(X_data,y_data,test_size=0.2,random_state=329)

display(X_trn.columns)
display(X_trn.shape, y_trn.shape)
display(X_vld.shape, y_vld.shape)
display(X_tst.shape, y_tst.shape)

Index(['BName', 'BName_sub', 'Author', 'Publshr', 'Author_mul', 'Pdate',
       'SalesPoint', 'Category'],
      dtype='object')

(101173, 8)

(101173,)

(25294, 8)

(25294,)

(31617, 8)

(31617,)

In [12]:
data_dict = {
    'trn': {
        'X': X_trn,
        'y': y_trn
        },
    'vld':{
        'X': X_vld,
        'y': y_vld
        },
    'tst':{
        'X': X_tst,
        'y': y_tst
        
    }
}

In [13]:
display(data_dict['trn']['X'].info())
display(data_dict['vld']['X'].info())
display(data_dict['tst']['X'].info())

<class 'pandas.core.frame.DataFrame'>
Index: 101173 entries, 96986 to 69788
Data columns (total 8 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   BName       101172 non-null  object
 1   BName_sub   5760 non-null    object
 2   Author      101173 non-null  object
 3   Publshr     101173 non-null  object
 4   Author_mul  101173 non-null  bool  
 5   Pdate       101173 non-null  int64 
 6   SalesPoint  101173 non-null  int64 
 7   Category    101173 non-null  object
dtypes: bool(1), int64(2), object(5)
memory usage: 6.3+ MB


None

<class 'pandas.core.frame.DataFrame'>
Index: 25294 entries, 132734 to 43660
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   BName       25294 non-null  object
 1   BName_sub   1434 non-null   object
 2   Author      25294 non-null  object
 3   Publshr     25294 non-null  object
 4   Author_mul  25294 non-null  bool  
 5   Pdate       25294 non-null  int64 
 6   SalesPoint  25294 non-null  int64 
 7   Category    25294 non-null  object
dtypes: bool(1), int64(2), object(5)
memory usage: 1.6+ MB


None

<class 'pandas.core.frame.DataFrame'>
Index: 31617 entries, 146028 to 115151
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   BName       31617 non-null  object
 1   BName_sub   1813 non-null   object
 2   Author      31617 non-null  object
 3   Publshr     31617 non-null  object
 4   Author_mul  31617 non-null  bool  
 5   Pdate       31617 non-null  int64 
 6   SalesPoint  31617 non-null  int64 
 7   Category    31617 non-null  object
dtypes: bool(1), int64(2), object(5)
memory usage: 2.0+ MB


None

In [14]:
mecab = Mecab()
tokenizer_basic = lambda x : mecab.morphs(x)
#cut_date = lambda x : [x[:2],x[2:4],x[4:6],x[6:]]
#apply tokenizer
cols_vec = ['Category','BName','BName_sub']
cols_freq = ['Author','Publshr']

In [15]:
for mode, data in data_dict.items():
    bookinfo = data['X']
    for col in cols_vec:
        bookinfo[col] = bookinfo[col].fillna('').apply(tokenizer_basic)
    #bookinfo['Pdate'] = bookinfo['Pdate'].astype(str).apply(cut_date)
    bookinfo[['Author_mul','Pdate']] = bookinfo[['Author_mul','Pdate']].astype(int).astype(str)
    data_dict[mode]['X'] = bookinfo

In [16]:
#아래 ths 는 EDA 결과 제가 자의적으로 정한 내용
bookinfo = data_dict['trn']['X']
ths_author = int(np.round(len(bookinfo)/500)*75)
ths_publshr = int(np.round(len(bookinfo)/500)*5)

map_author_encode = make_author_encode_map(bookinfo[['Author','SalesPoint']],ths_author)
map_publshr_encode = make_publshr_encode_map(bookinfo['Publshr'],ths_publshr)

encode_maps = {
    'Author' : lambda x : encode_tokens(map_author_encode,x,oov=False),
    'Publshr' : lambda x : encode_tokens(map_publshr_encode,x,oov=False),
}

In [17]:
for col in tqdm(cols_freq):
    bookinfo[col] = bookinfo[col].map(encode_maps[col]).astype(str)

100%|██████████| 2/2 [00:00<00:00, 13.71it/s]


In [18]:
#make encoding map
cols_tknz= ['BName', 'BName_sub', 'Author', 'Publshr', 'Author_mul', 'Pdate','Category']
book_tknzed = bookinfo[cols_tknz].to_dict('series')
#book_name, book_subname, category = book_tknzed['BName'], book_tknzed['BName_sub'],book_tknzed['Category']

In [19]:
display('268' in book_tknzed['Publshr'].values)
display(book_tknzed['Publshr'].values)

True

array(['19', '92', '37', ..., '392', '0', '0'], dtype=object)

In [20]:
cols_num = list(filter(lambda x : x not in cols_vec, cols_tknz))
number_slicer = lambda x : [x[max(0,i-2):i] for i in range(len(x),0,-2)[::-1]]
for col in cols_num:
    print(col)
    book_tknzed[col] = book_tknzed[col].apply(number_slicer)

Author
Publshr
Author_mul
Pdate


In [48]:
book_tknzed['Publshr']

96986        [19]
48605        [92]
30428        [37]
110065       [65]
124342    [1, 62]
           ...   
128584        [1]
10037     [1, 68]
70829     [3, 92]
79290         [0]
69788         [0]
Name: Publshr, Length: 101173, dtype: object

In [49]:
tkn_vals=[]
for col in cols_num:
    tkn_vals.append(np.array(list(itertools.chain(*book_tknzed[col].values))))
for col in cols_vec:
    temp = pd.Series(list(itertools.chain(*book_tknzed[col].values)))
    cond = ~(pd.to_numeric(temp,errors='coerce')).isna()
    temp[cond] = temp[cond].apply(number_slicer)
    temp[~cond] = temp[~cond].apply(lambda x :[x])
    tkn_vals.append(np.array(list(itertools.chain(*temp.values))))
    
    
#tokens = np.concatenate(tkn_vals,axis=0)
#tokens
tkn_vals

[array(['0', '0', '0', ..., '0', '0', '0'], dtype='<U2'),
 array(['19', '92', '37', ..., '92', '0', '0'], dtype='<U2'),
 array(['1', '0', '0', ..., '1', '0', '0'], dtype='<U1'),
 array(['20', '15', '05', ..., '11', '03', '21'], dtype='<U2'),
 array(['소설', '/', '시', ..., '/', '전문', '서적'], dtype='<U3'),
 array(['니세코', '이', '비하인드', ..., '원리', '와', '이론'], dtype='<U15'),
 array(['(', '전', '작품', ..., '특별', '판', ')'], dtype='<U13')]

In [50]:
tokens = np.concatenate(tkn_vals,axis=0)

In [51]:
tokens_ds = pd.Series(tokens)
cond_numeric = ~(pd.to_numeric(tokens_ds,errors='coerce')).isna()

tokens_num = tokens[cond_numeric]
tokens_str = tokens[~cond_numeric]


In [52]:
print(len(tokens_num),len(tokens_str))
print(len(np.unique(tokens_num)),len(np.unique(tokens_str)))

824863 822353
110 31441


In [53]:
temp,cnt = np.unique(tokens_num,return_counts=True)
temp2 = np.apply_along_axis(lambda x : [(len(n),cnt[i]) for i,n in enumerate(x)],axis=0,arr=temp)
temp2

array([[     1, 157455],
       [     2,   6217],
       [     2,  22521],
       [     2,  13736],
       [     2,  14905],
       [     2,  13936],
       [     2,  17652],
       [     2,  14136],
       [     2,  15021],
       [     2,  13977],
       [     2,  14020],
       [     1,  68256],
       [     2,  25577],
       [     2,  17373],
       [     2,  16984],
       [     2,   8453],
       [     2,   8460],
       [     2,  14468],
       [     2,   8385],
       [     2,   8649],
       [     2,   8872],
       [     2,  16041],
       [     1,  19417],
       [     2, 116666],
       [     2,   8538],
       [     2,   8723],
       [     2,   8377],
       [     2,   6275],
       [     2,   8251],
       [     2,   3384],
       [     2,   3847],
       [     2,   4150],
       [     2,   3310],
       [     1,  13618],
       [     2,   8444],
       [     2,   3357],
       [     2,   1069],
       [     2,   1027],
       [     2,   1048],
       [     2,    970],


In [54]:
np.sum(temp2[:,1][temp2[:,0] < 3])/np.sum(cnt)

1.0

In [55]:
tkns_num = 31000
token_freq_str = np.unique(tokens_str,return_counts=True)
token_freq_num = np.unique(tokens_num,return_counts=True)
map_token_encode_str = make_encoding_by_freq(token_freq_str,size_feat=tkns_num)
map_token_encode_num = make_encoding_by_freq(token_freq_num)

In [56]:
token_freq_num

(array(['0', '00', '01', '02', '03', '04', '05', '06', '07', '08', '09',
        '1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19',
        '2', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29',
        '3', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39',
        '4', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49',
        '5', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59',
        '6', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69',
        '7', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79',
        '8', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89',
        '9', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99'],
       dtype='<U15'),
 array([157455,   6217,  22521,  13736,  14905,  13936,  17652,  14136,
         15021,  13977,  14020,  68256,  25577,  17373,  16984,   8453,
          8460,  14468,   8385,   8649,   8872,  16041,  19417, 116666,
          8538,   8723,   8377,

In [57]:
map_token_encode_num_placed = dict()
for key,val in map_token_encode_num.items():
    if key == '[PAD]' : continue
    map_token_encode_num_placed[key] = val+tkns_num

map_token_encode_total = dict(map_token_encode_str,**map_token_encode_num_placed)

In [58]:
encode_1line =lambda x: list(map(lambda y : encode_tokens(map_token_encode_total,y),x))
encode_1line_num =lambda x: list(map(lambda y : encode_tokens(map_token_encode_num,y),x))

In [59]:
display(encode_1line(['68','268','2341252345','askdlfjwef']))
display(encode_1line_num(['68','268','2341252345','askdlfjwef']))

[31098, 31112, 31112, 31112]

[98, 112, 112, 112]

In [60]:
maxlens={
    'Category' : 5,
    'BName' : 27,
    'BName_sub' : 23,
    'Pdate' : 4,
    'Publshr' : 2,
    'Author' : 2,
    'Author_mul' : 1
}
x_cols = ['BName', 'BName_sub', 'Author',
   'Author_mul', 'Publshr', 'Pdate','Category']

book_cols = ['BName', 'BName_sub', 'Author', 'Author_mul', 'Publshr', 'Pdate',
       'RglPrice', 'SalesPoint', 'Category']
xcols_scalar = list(filter(lambda x : x not in maxlens.keys(),x_cols)) 

In [61]:
from module_aladin.nlp import erase_space
def find_split_num(text):
    pat = r'[\s\d]+'
    mtch_list = list(re.finditer(pat,text))
    c,rslt=0,list()
    for m in mtch_list:
        if c < m.start() : rslt.append((text[c:m.start()],0))
        rslt.append((erase_space(text[m.start():m.end()]),1))
        c=m.end()
    else :
        if c < len(text): rslt.append((text[c:len(text)],0))
    return rslt


def number_splitter(data):
    rslt = list()
    for txt in data:
        temp = find_split_num(txt)
        if not temp :rslt.append(temp)
        else :
            intrmd=list()
            for ele in temp:
                if ele[1] == 0 : intrmd.append(ele[0])
                else : intrmd.extend(number_slicer(ele[0]))
            rslt.extend(intrmd)
    return rslt        

In [62]:

#encode X
X_encoded=dict()
for mode,sample in tqdm(data_dict.items()):
    X_mode = sample['X'].copy()
    #padding and encoding
    encoded = pd.DataFrame(index=X_mode.index) 
    for col in tqdm(x_cols):
        if col in cols_num:
            X_mode[col] = X_mode[col].apply(number_slicer)
        if col in cols_vec:
            X_mode[col] = X_mode[col].apply(number_splitter)
        
        padded = pad_sequences(X_mode[col],padding='post',
                                   maxlen=maxlens[col],
                                   value='[PAD]',dtype=object)
        encoded[col] = list(np.apply_along_axis(encode_1line,0,padded))

    concat_tknzed =np.apply_along_axis(np.hstack,1,encoded[list(maxlens.keys())].values)
    x_scalar = encoded[xcols_scalar].values
    X = np.hstack((concat_tknzed,x_scalar))
    X_encoded[mode] = X    

100%|██████████| 3/3 [00:13<00:00,  4.36s/it]


In [63]:
data_dict['trn']['X'].iloc[74310]

BName         [눈, 이, 아닌, 것, 으로, 도, 읽, 은, 기분]
BName_sub                                 []
Author                                     0
Publshr                                  268
Author_mul                                 0
Pdate                               20171230
SalesPoint                               666
Category                               [에세이]
Name: 114793, dtype: object

In [64]:
X_encoded['trn'][74310]

array([2.2000e+01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       4.3100e+02, 1.5000e+01, 1.3470e+03, 9.7000e+01, 6.9000e+01,
       7.5000e+01, 9.1000e+01, 1.1000e+01, 1.9320e+03, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       3.1002e+04, 3.1023e+04, 3.1009e+04, 3.1027e+04, 3.1006e+04,
       3.1098e+04, 3.1001e+04, 0.0000e+00, 3.1001e+04])

In [65]:
data_dict['trn']['X'].iloc[74310,3], '268' in data_dict['trn']['X']['Publshr'].values

('268', True)

In [66]:
map_token_encode_num['268']

KeyError: '268'

In [67]:
display(np.sum(np.isnan(X_encoded['trn'].astype(np.float64))))
display(np.sum(np.isnan(X_encoded['vld'].astype(np.float64))))
display(np.sum(np.isnan(X_encoded['tst'].astype(np.float64))))

0

0

0

In [68]:
print(len(np.unique(X_encoded['trn'])))
print(np.max(X_encoded['trn']))
print(np.min(X_encoded['trn']))

31095
31112.0
0.0


In [69]:
X_coded = {
    mode : data.astype(np.int32)
    for mode, data in X_encoded.items()
} 

In [73]:
from collections import defaultdict
y_coded = defaultdict(dict)
for mode,sample in data_dict.items():
    y_coded[mode]['value'] = sample['y'].to_numpy()
    temp = sample['y'].astype(str).apply(lambda x : number_slicer(x)[::-1])
    padded = pad_sequences(temp,padding='post',
                                   maxlen=5,
                                   value='[PAD]',dtype=object)
    intrmd = np.apply_along_axis(encode_1line_num,0,padded)
    y_coded[mode]['coded'] = intrmd.astype(np.int32)
    y_coded[mode]['decode_map'] = {
        v:k
        for k,v in map_token_encode_num.items()}
    y_coded[mode]['freq'] = np.unique(intrmd.astype(np.int32),return_counts=True)
    

In [74]:
y_coded['trn']

{'value': array([ 6800, 11900, 20000, ..., 14000,  7000, 20000]),
 'coded': array([[ 34,  98,   0,   0,   0],
        [ 34,  10,   3,   0,   0],
        [ 34,  34,   6,   0,   0],
        ...,
        [ 34,  56,   3,   0,   0],
        [ 34, 105,   0,   0,   0],
        [ 34,  34,   6,   0,   0]], dtype=int32),
 'decode_map': {1: '0',
  2: '20',
  3: '1',
  4: '10',
  5: '01',
  6: '2',
  7: '05',
  8: '11',
  9: '12',
  10: '19',
  11: '07',
  12: '03',
  13: '15',
  14: '06',
  15: '09',
  16: '08',
  17: '04',
  18: '02',
  19: '3',
  20: '4',
  21: '18',
  22: '22',
  23: '17',
  24: '21',
  25: '14',
  26: '13',
  27: '30',
  28: '16',
  29: '23',
  30: '25',
  31: '5',
  32: '6',
  33: '24',
  34: '00',
  35: '7',
  36: '8',
  37: '9',
  38: '28',
  39: '27',
  40: '26',
  41: '31',
  42: '29',
  43: '99',
  44: '98',
  45: '97',
  46: '96',
  47: '95',
  48: '32',
  49: '93',
  50: '34',
  51: '39',
  52: '37',
  53: '36',
  54: '94',
  55: '33',
  56: '40',
  57: '50',
  58: '4

In [75]:
data_type = 'encodedXy'
strat=0
ver=2.0
dir_path = os.path.join(RSLT_DIR,'model_input')
for mode, x_scaled in X_coded.items():
    save_pkl(dir_path,'{}.v{}_st-{}_X_{}.pkl'.format(data_type,ver,strat,mode),x_scaled)
for mode,data in y_coded.items(): 
    save_pkl(dir_path,'{}.v{}_st-{}_y_{}.pkl'.format(data_type,ver,strat,mode),data)